In [65]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
import re

In [183]:
from google.colab import files
uploaded = files.upload()

Saving emails.csv to emails.csv


In [184]:
df = pd.read_csv('emails.csv')

In [185]:
df.head(5)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [186]:
df.shape

(5728, 2)

In [188]:
#check for duplicates and remove them
df.drop_duplicates(inplace=True)
df.shape

(5695, 2)

In [189]:
df.isnull().sum()

text    0
spam    0
dtype: int64

In [190]:
df['spam'].value_counts()

0    4327
1    1368
Name: spam, dtype: int64

In [191]:
#download stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [192]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
def process_text(text):
  text = re.sub(r"http\S+", "", text) #remove hypperlinks
  text = text.lower() #convert to lowercase
  text = re.sub(r'\d+', '', text) #remove numbers
  text = text.translate(str.maketrans(dict.fromkeys(string.punctuation))) #remove punctuation
  text = text.strip()#remove whitespace
  text = text.replace('\n','') #replace new line with space
  text = [word for word in text.split() if not word in stopwords.words('english')] #remove stopwords and tokenize
  text = [lemmatizer.lemmatize(word) for word in text] #lemmatize the words
  return text

In [193]:
df['text'].head().apply(process_text)

0    [subject, naturally, irresistible, corporate, ...
1    [subject, stock, trading, gunslinger, fanny, m...
2    [subject, unbelievable, new, home, made, easy,...
3    [subject, color, printing, special, request, a...
4    [subject, money, get, software, cd, software, ...
Name: text, dtype: object

In [203]:
bow = CountVectorizer(analyzer=process_text).fit_transform(df['text']) #bag of words
bow.shape


(5695, 31001)

In [195]:
#split data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(bow, df['spam'], test_size=0.20, random_state=0 )

In [196]:
#build classifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(x_train,y_train) #train classifier
y_pred = classifier.predict(x_test) #test classifier

In [200]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))
print('accuracy', accuracy_score(y_test,y_pred))
print('precision',metrics.precision_score(y_test,y_pred))
print('recall',metrics.recall_score(y_test,y_pred))


[[862   8]
 [  3 266]]
accuracy 0.990342405618964
precision 0.9708029197080292
recall 0.9888475836431226
